# Adding small communities to de Boer (2000)

Having recreated and validated found results of de Boer (2000) together with a less ad hoc variant of the Bark Operator we shift our focus on extending de Boer (2000).
As we discuss in the report, this extension is in changing the interaction network of the agents.
Instead of randomly picking a speaker and and imitator, we use a selection strategy based on a small community.
We have different roles in this small community, which each have their own characteristic properties.


**References**

de Boer, B. (2000). Self-organization in vowel systems. *In Journal of Phonetics (Vol. 28, Issue 4, pp. 441–465)*. Elsevier BV. https://doi.org/10.1006/jpho.2000.0125


<hr>
<hr>

## Table of contents

- Student information
- Required imports
- Adding an extra CommunityRole enum and CommunityBehaviour class
- Extending the agent class
- Creating custom game engine

<hr>
<hr>

## Student information

- **Name**: Bontinck Lennert
- **Student ID**: 568702
- **Affiliation**: VUB - Master Computer Science: AI

<hr>
<hr>

## Required imports

Below we import the required things for this notebook.

In [9]:
# Import imitation game classes made in the previous notebook
from imitationGameClasses import Agent, Synthesizer, BarkOperator, GameState

# Used for saving and loading variables
import pickle;

# Used for easier numerical operations
import numpy as np;
import math
import random as rnd;

# Enum for role
from enum import Enum

<hr>
<hr>

## Adding an extra CommunityRole enum and CommunityBehaviour class

In order to represent the role of the community member (agent), we make a simple enum type.  



In [10]:
class CommunityRole(Enum):
    """This is an enum used to represent the agent's community role."""
    PROFESSOR = 1;
    DOCTERATE = 2;
    STUDENT = 3;
    BABY = 4;
    PARENT = 5;
    GRANDPARENT = 6;
    MYPARENT = 7;
    MYGRANDPARENT = 8;

To represent the influence and other community based parameters for an agent we add a CommunityInfluence class.

In [11]:
class CommunityBehaviour():
    """This is a class used to represent the innfluence and other community based parameters for an agent."""
    def __init__(self, new_sound_prob: float, phoneme_step_size_dictionary: float,
                 synthesizer: Synthesizer, influential_agent_types: list):
        """Creates a CommunityBehaviour instance."""
        # Chance of adding new random vowel
        self.new_sound_prob = new_sound_prob;
        
        # Step size control
        self.phoneme_step_size_dictionary = phoneme_step_size_dictionary;
        
        # Noise control
        self.synthesizer = synthesizer;
        
        # Types of agents that can influence immitator to create new vowel
        self.influential_agent_types = influential_agent_types;

<hr>
<hr>

## Extending the agent class



In [12]:
# Defining child class so that Agent methods are copied
class CommunityAgent(Agent):
      
    # Constructor
    def __init__(self, synthesizer: Synthesizer, bark_operator: BarkOperator,
                    community_role : CommunityRole, community_behaviour: CommunityBehaviour,
                    logger: bool = False,
                    phoneme_step_size: float = 0.1, max_similar_sound_loops: int = 20, max_semi_random_loop: int = 5,
                    sound_threshold_game: float = 0.5, sound_threshold_agent:float = 0.7, sound_minimum_tries: int = 5,
                    cleanup_prob = 0.1, new_sound_prob = 0.01, merge_prob = 1      
                ):
        
        # Use init of Agent
        Agent.__init__(self, synthesizer = synthesizer, bark_operator = bark_operator,
                       logger = logger, phoneme_step_size = phoneme_step_size,
                       max_similar_sound_loops = max_similar_sound_loops,
                       max_semi_random_loop = max_semi_random_loop, sound_threshold_game = sound_threshold_game,
                       sound_threshold_agent = sound_threshold_agent, sound_minimum_tries = sound_minimum_tries,
                       cleanup_prob = cleanup_prob, new_sound_prob = new_sound_prob,
                       merge_prob = merge_prob);
        
        # Store community role
        self.community_role = community_role;
        
        # Store community behaviour
        self.community_behaviour = community_behaviour;
        
    # Prepare for the current game
    def prepare_current_game(self, oponent_role: CommunityRole):
        """Prepare parameters to reflect oponent of current game."""
        # Step size dependent on oponent
        self.phoneme_step_size = self.community_behaviour.phoneme_step_size_dictionary[oponent_role];
        
        # Save role of oponent for game
        self.oponent_role = oponent_role;
        
    # Prepare for the current game
    def change_agent_role_and_behaviour(self, new_role: CommunityRole, new_behaviour: CommunityBehaviour):
        """Prepare parameters to reflect oponent of current game."""
        # Store community role
        self.community_role = new_role;
        
        # Store community behaviour
        self.community_behaviour = new_behaviour;
        
        if self.logger:
            print(f"{self.name}: update my role and behaviour to reflect {new_role.name}.");

        

    # Edit so that we only change our vowel repetoire if we "care about" the oponent
    def process_non_verbal_imitation_confirmation(self, was_success):
        """Processes the non verbal confirmation if an imitation was correct, ending the game cycle."""
        if was_success:
            # Save success
            self.known_sounds[self.last_spoken_sound].was_success();
            # "Shift closer" if we care about oponent agent
            if self.oponent_role in self.community_behaviour.influential_agent_types:
                improved_sound = self.improve_sound(self.known_sounds[self.last_spoken_sound], self.last_heard_utterance);
                self.known_sounds[self.last_spoken_sound].improve(improved_sound);
        else:
            if self.known_sounds[self.last_spoken_sound].success_ratio() < self.sound_threshold_game:
                # Probably bad sound - "Shift closer" and we care about oponent
                if self.oponent_role in self.community_behaviour.influential_agent_types:
                    improved_sound = self.improve_sound(self.known_sounds[self.last_spoken_sound], self.last_heard_utterance);
                    self.known_sounds[self.last_spoken_sound].improve(improved_sound);
            else:
                # Probably good sound - add new sound to repetoire if we care about oponent
                if self.oponent_role in self.community_behaviour.influential_agent_types:
                    self.add_similar_sound(self.last_heard_utterance);


        if self.logger:
            if was_success:
                print(self.name + ": had a confirmed match, changed my sound to match closer.");

        # End of current game
        self.prepare_for_new_game(was_imitator=True, was_succes= was_success);


<hr>
<hr>

## Creating custom game engine

We now create a custom game engine to play the more complex game.
For this we need instances of the previously added CommunityBehaviour class for each type of agent.

In [19]:
class CommunityGameEngine:
    """This is a class used to represent an imitation game egine."""
    def __init__(self,
                 community_member_amounts: dict,
                 community_behaviours: dict,
                 agent_age_group_width: int,
                 
                 iterations: int, bark_operator: BarkOperator, 
                 agent_sound_threshold_game: float = 0.5, agent_sound_threshold_self:float = 0.7,
                 agent_sound_minimum_tries: int = 5):
        """Creates a Community Game Engine instance for the provided community settings."""
        
        # Keep track of number of agents
        self.community_member_amounts = community_member_amounts;
        
        total_amount_of_agents = 0;
        for community_role in community_member_amounts:
            total_amount_of_agents += community_member_amounts[community_role];
        
        # Keep track of community behaviours
        self.community_behaviours = community_behaviours;
        
        # Keep track of number of iterations
        self.iterations = iterations;
        
        # Keep track of age group width
        self.agent_age_group_width = agent_age_group_width;
        
        # Keep track of bark operator etc
        self.bark_operator = bark_operator;
        self.agent_sound_threshold_game = agent_sound_threshold_game;
        self.agent_sound_threshold_self = agent_sound_threshold_self;
        self.agent_sound_minimum_tries = agent_sound_minimum_tries;

        # Create the agents
        self.agents = [];
                
        for community_role in community_member_amounts:
            self.agents += [CommunityAgent(synthesizer= community_behaviours[community_role].synthesizer,
                                           bark_operator= bark_operator, 
                                           community_role = community_role,
                                           community_behaviour = community_behaviours[community_role],
                                           sound_threshold_game= agent_sound_threshold_game,
                                           sound_threshold_agent= agent_sound_threshold_self,
                                           sound_minimum_tries= agent_sound_minimum_tries,
                                           new_sound_prob = community_behaviours[community_role].new_sound_prob)
                            for n in range(community_member_amounts[community_role])];
            
        # Keep track of parents of agents
        # TODO: needs init :)
        self.parent_tree = {};
            
    def __play_one_agent_pair(self, speaker: CommunityAgent, imitator: CommunityAgent):
        # prepare agents
        speaker.prepare_current_game(imitator.community_role);
        imitator.prepare_current_game(speaker.community_role);
        
        # play game
        start_utterance = speaker.say_something();
        imitated_utterance = imitator.imitate_sound(start_utterance);
        validation = speaker.validate_imitation(imitated_utterance);
        imitator.process_non_verbal_imitation_confirmation(validation);
        

    def __play_full_agent_aging_round(self):
        """Plays an imitation game round where all agents play so that age evolves constant across agents. """
        # Chose pairs such that each agent is a listener at least once
        for community_role in self.community_member_amounts:
            # Determine all agents of that type
            agents_of_type = [a for a in self.agents if a.community_role == community_role];
            
            # Determine to whom the agents of that type may listen
            possible_speakers = [a for a in self.agents if a.community_role in self.community_behaviours[community_role].influential_agent_types]
                
            # Find all agents of that role and play game as imitator
            for imitator in agents_of_type:
                # Find a speaker from the preffered speakers, if there is such a list (e.g. not for babies since MYPARENT)
                if (possible_speakers):
                    speaker = rnd.choice(possible_speakers);
                
                # If "my parent" or "my grandparent" and ok odds
                print(imitator.community_behaviour);
                if 1/(len(possible_speakers)+1) >= rnd.random() and CommunityRole.MYPARENT in imitator.community_behaviour.influential_agent_types:
                    speaker = self.parent_tree[imitator];
                    
                if 1/(len(possible_speakers)+1) >= rnd.random() and CommunityRole.MYGRANDPARENT in imitator.community_behaviour.influential_agent_types:
                    speaker = self.parent_tree[self.parent_tree[imitator]];
                    
                # Play game
                # TODO: remove
                print(f"me, a {imitator.community_role}, is listining to a {speaker.community_role}");
                self.__play_one_agent_pair(speaker, imitator);
        
    def play_imitation_game(self, checkpoints: list):
        """Plays an imitation game and returns a vector of GameState objects.
        - checkpoints: list of iteration numbers at which the state of the game should be saved (after playing that iteration)."""
        
        game_states = [None] * len(checkpoints);

        for i in range(self.iterations):
            # Play one iteration of the game
            self.__play_full_agent_aging_round();

            # After playing the games, check if checkpoint reached for storing
            if i + 1 in checkpoints:
                # Force merge of agent for Energy measure
                for agent in self.agents:
                    agent.merge_similar_sound();
                    
                # Store imitation game state
                game_states[checkpoints.index(i + 1)] = GameState(self.agents, i + 1);
            
            # Check if aging round (e.g. move one step up in hierarchy)
            if i % self.agent_age_group_width == 0 and i != 0:
                # Kill professors and grandparents
                dead_agents = [a for a in self.agents if a.community_role in [CommunityRole.GRANDPARENT, CommunityRole.PROFESSOR]];
                for dead_agent in dead_agents:
                    del self.parent_tree[dead_agent];
                    self.agents.remove(dead_agent);
                    
                # make docterates professors
                docterates = [a for a in self.agents if a.community_role == CommunityRole.DOCTERATE];
                for docterate in docterates:
                    docterate.change_agent_role_and_behaviour(new_role = CommunityRole.PROFESSOR, 
                                                              new_behaviour = self.community_behaviours[CommunityRole.PROFESSOR]);
                    
                
                # make parents grandparents
                parents = [a for a in self.agents if a.community_role == CommunityRole.PARENT];
                for parent in parents:
                    parent.change_agent_role_and_behaviour(new_role = CommunityRole.GRANDPARENT, 
                                                           new_behaviour = self.community_behaviours[CommunityRole.GRANDPARENT]);
                    
                # make students either docterate or parent depending on parent
                students = [a for a in self.agents if a.community_role == CommunityRole.STUDENT];
                for student in students:
                    if self.parent_tree[student].community_role in [CommunityRole.DOCTERATE, CommunityRole.PROFESSOR]:
                        student.change_agent_role_and_behaviour(new_role = CommunityRole.DOCTERATE,
                                                                new_behaviour = self.community_behaviours[CommunityRole.DOCTERATE]);
                        
                    if self.parent_tree[student].community_role in [CommunityRole.PARENT, CommunityRole.GRANDPARENT]:
                        student.change_agent_role_and_behaviour(new_role = CommunityRole.PARENT,
                                                                new_behaviour = self.community_behaviours[CommunityRole.PARENT]);
                
                # make baby student
                babies = [a for a in self.agents if a.community_role == CommunityRole.BABY];
                for baby in babies:
                    baby.change_agent_role_and_behaviour(new_role = CommunityRole.STUDENT,
                                                         new_behaviour = self.community_behaviours[CommunityRole.STUDENT]);
                    
                # create new babies, one for each parent
                new_parents = [a for a in self.agents if a.community_role in [CommunityRole.PARENT, CommunityRole.DOCTERATE]];
                for parent in new_parents:
                    new_baby = [CommunityAgent(synthesizer= self.community_behaviours[CommunityRole.BABY].synthesizer,
                                               bark_operator= self.bark_operator, 
                                               community_role = CommunityRole.BABY,
                                               community_behaviour = self.community_behaviours[CommunityRole.BABY],
                                               sound_threshold_game= self.agent_sound_threshold_game,
                                               sound_threshold_agent= self.agent_sound_threshold_self,
                                               sound_minimum_tries= self.agent_sound_minimum_tries,
                                               new_sound_prob = self.community_behaviours[CommunityRole.BABY].new_sound_prob)];
                                        
                    # Store new baby and its parent
                    self.agents += new_baby;
                    self.parent_tree[new_baby] = parent;

        # Return the game states
        return game_states;

In [20]:
bark_operator = BarkOperator(critical_distance = 3.5, second_formant_weight = 0.3);
community_member_amounts = {
    CommunityRole.BABY: 5,
    CommunityRole.STUDENT: 5,
    CommunityRole.DOCTERATE: 2,
    CommunityRole.PROFESSOR: 2,
    CommunityRole.PARENT: 3,
    CommunityRole.GRANDPARENT: 3,
    }

community_behaviours = {
    CommunityRole.BABY: CommunityBehaviour(new_sound_prob = 0.1,
                                           synthesizer = Synthesizer(max_noise_ambient = 0.4),
                                           influential_agent_types = [CommunityRole.MYPARENT, CommunityRole.MYGRANDPARENT],
                                           phoneme_step_size_dictionary= {
                                               CommunityRole.BABY: 0.15,
                                               CommunityRole.STUDENT: 0.15,
                                               CommunityRole.DOCTERATE: 0.15,
                                               CommunityRole.PROFESSOR: 0.15,
                                               CommunityRole.PARENT: 0.15,
                                               CommunityRole.GRANDPARENT: 0.15,
                                               }),
    CommunityRole.STUDENT: CommunityBehaviour(new_sound_prob = 0.02,
                                              synthesizer = Synthesizer(max_noise_ambient = 0.15),
                                              influential_agent_types = [CommunityRole.MYPARENT, CommunityRole.MYGRANDPARENT, CommunityRole.STUDENT, CommunityRole.PROFESSOR],
                                              phoneme_step_size_dictionary= {
                                                  CommunityRole.BABY: 0,
                                                  CommunityRole.STUDENT: 0.01,
                                                  CommunityRole.DOCTERATE: 0.05,
                                                  CommunityRole.PROFESSOR: 0.05,
                                                  CommunityRole.PARENT: 0.02,
                                                  CommunityRole.GRANDPARENT: 0.02,
                                                  }),
    CommunityRole.DOCTERATE: CommunityBehaviour(new_sound_prob = 0.005,
                                                synthesizer = Synthesizer(max_noise_ambient = 0.05),
                                                influential_agent_types = [CommunityRole.MYPARENT, CommunityRole.DOCTERATE, CommunityRole.PROFESSOR],
                                                phoneme_step_size_dictionary= {
                                                    CommunityRole.BABY: 0,
                                                    CommunityRole.STUDENT: 0,
                                                    CommunityRole.DOCTERATE: 0.015,
                                                    CommunityRole.PROFESSOR: 0.025,
                                                    CommunityRole.PARENT: 0.015,
                                                    CommunityRole.GRANDPARENT: 0,
                                                    }),
    CommunityRole.PROFESSOR: CommunityBehaviour(new_sound_prob = 0.001,
                                                synthesizer = Synthesizer(max_noise_ambient = 0.01),
                                                influential_agent_types = [CommunityRole.PROFESSOR],
                                                phoneme_step_size_dictionary= {
                                                    CommunityRole.BABY: 0,
                                                    CommunityRole.STUDENT: 0,
                                                    CommunityRole.DOCTERATE: 0,
                                                    CommunityRole.PROFESSOR: 0.015,
                                                    CommunityRole.PARENT: 0,
                                                    CommunityRole.GRANDPARENT: 0,
                                                    }),
    CommunityRole.PARENT: CommunityBehaviour(new_sound_prob = 0.01,
                                             synthesizer = Synthesizer(max_noise_ambient = 0.1),
                                             influential_agent_types = [CommunityRole.MYPARENT, CommunityRole.PARENT],
                                             phoneme_step_size_dictionary= {
                                                 CommunityRole.BABY: 0,
                                                 CommunityRole.STUDENT: 0,
                                                 CommunityRole.DOCTERATE: 0,
                                                 CommunityRole.PROFESSOR: 0,
                                                 CommunityRole.PARENT: 0.015,
                                                 CommunityRole.GRANDPARENT: 0.015,
                                                 }),
    CommunityRole.GRANDPARENT: CommunityBehaviour(new_sound_prob = 0.005,
                                             synthesizer = Synthesizer(max_noise_ambient = 0.1),
                                             influential_agent_types = [CommunityRole.GRANDPARENT],
                                             phoneme_step_size_dictionary= {
                                                 CommunityRole.BABY: 0,
                                                 CommunityRole.STUDENT: 0,
                                                 CommunityRole.DOCTERATE: 0,
                                                 CommunityRole.PROFESSOR: 0,
                                                 CommunityRole.PARENT: 0,
                                                 CommunityRole.GRANDPARENT: 0.015,
                                                 }),
    }

community_game_engine = CommunityGameEngine(community_member_amounts = community_member_amounts,
                                            community_behaviours = community_behaviours,
                                            agent_age_group_width = 200,
                                            iterations = 10000,
                                            bark_operator = bark_operator);

In [21]:
checkpoints = [20, 500, 2000, 10000];
results = community_game_engine.play_imitation_game(checkpoints);

KeyError: <__main__.CommunityAgent object at 0x7f9e5bc244c0>